In [10]:
access_token = "sl.BuH4FD4CoaTOSEnioggSVRFLFjiZqmcp_DN6B_ERhSvyM_sj1PHedrbdCxvYKDWzvJWKHeGrznpg3k76jdfYvYfjb3LWuzQ7wm_GaZDpJQeGxamI545ZErGC8APkh9kueI3sg_xRzYJe6NCPSwfP"

In [15]:
import dropbox

class TransferData:
    def __init__(self, access_token):
        self.access_token = access_token

    def upload_file(self, file_from, file_to):
        """upload a file to Dropbox using API v2
        """
        dbx = dropbox.Dropbox(self.access_token)

        with open(file_from, 'rb') as f:
            dbx.files_upload(f.read(), file_to)

    def download_file(self, file_from, file_to):
        """Download a file from Dropbox to a local file."""
        dbx = dropbox.Dropbox(self.access_token)

        try:
            # Download the file from Dropbox
            metadata, response = dbx.files_download(file_from)

            # Save the file locally
            with open(file_to, 'wb') as f:
                f.write(response.content)

            print(f"File downloaded successfully: {file_to}")

        except dropbox.exceptions.ApiError as err:
            print(f"Error downloading file: {err}")


    


In [11]:
transferData = TransferData(access_token)

file_from = 'vit_pipeline.py'
file_to = '/test.py'  # The full path to upload the file to, including the file name

# API v2
transferData.upload_file(file_from, file_to)

In [16]:
transferData = TransferData(access_token)

file_from = 'vit_pipeline123.py'
file_to = '/test.py'  # The full path to upload the file to, including the file name

transferData.download_file(file_to, file_from)

File downloaded successfully: vit_pipeline123.py


In [8]:

import dropbox

client = dropbox.client.DropboxClient(access_token)
print('linked account: ', client.account_info())


AttributeError: module 'dropbox' has no attribute 'client'

In [ ]:

f = open('working-draft.txt', 'rb')
response = client.put_file('/magnum-opus.txt', f)
print 'uploaded: ', response

folder_metadata = client.metadata('/')
print 'metadata: ', folder_metadata

f, metadata = client.get_file_and_metadata('/magnum-opus.txt')
out = open('magnum-opus.txt', 'wb')
out.write(f.read())
out.close()
print metadata

In [1]:
from vit_pipeline import *

import os
import torch
import torch.utils.data
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pathlib
import typing

import context_handlers
import models
import utils
import data_preprocessing
from ltn_support import *

In [2]:
fine_tuners, loaders, devices, num_fine_grain_classes, num_coarse_grain_classes = initiate(
    pretrained_path="//Users/khoavo2003/cs224/metacognitive_error_detection_and_correction_v2/model/vit_b_16_lr0.0001_LTN_soft_marginal_beta_0.5_batch_size_512_step_size_1_scheduler_gamma_0.5.pth",
    combined=True,
    train=False,
    debug=False,
    lrs=0.0001)

Models: ['vit_b_16']
Epochs num: 5
Learning rates: 0.0001
Total number of train images: 7823
Total number of test images: 1621
Using mps
Loading pretrained model from //Users/khoavo2003/cs224/metacognitive_error_detection_and_correction_v2/model/vit_b_16_lr0.0001_LTN_soft_marginal_beta_0.5_batch_size_512_step_size_1_scheduler_gamma_0.5.pth


In [3]:
fine_tuner = fine_tuners[0]
device = devices[0]

logits_to_predicate = ltn.Predicate(LogitsToPredicate()).to(ltn.device)

test_loader = loaders['test']
fine_tuner.to(device)
fine_tuner.eval()

test_fine_prediction = []
test_coarse_prediction = []

test_fine_ground_truth = []
test_coarse_ground_truth = []

name_list = []

print(f'Testing {fine_tuner} on {device}...')

with torch.no_grad():
    if utils.is_local():
        from tqdm import tqdm
        gen = tqdm(enumerate(test_loader), total=len(test_loader))
    else:
        gen = enumerate(test_loader)

    for i, data in gen:
        X, Y_fine_grain, names, Y_coarse_grain = data[0].to(device), data[1].to(device), data[2], data[3].to(device)

        Y_pred = fine_tuner(X)
        
        if i == 0:
            Y_pred_all = Y_pred
            Y_fine_grain_all = Y_fine_grain
            Y_coarse_grain_all = Y_coarse_grain
        else:
            Y_pred_all = torch.cat([Y_pred, Y_pred_all], dim=0)
            Y_fine_grain_all = torch.cat([Y_fine_grain, Y_fine_grain_all], dim=0)
            Y_coarse_grain_all = torch.cat([Y_coarse_grain, Y_coarse_grain_all], dim=0)

        Y_pred_fine_grain = Y_pred[:, :len(data_preprocessing.fine_grain_classes)]
        Y_pred_coarse_grain = Y_pred[:, len(data_preprocessing.fine_grain_classes):]

        predicted_fine = torch.max(Y_pred_fine_grain, 1)[1]
        predicted_coarse = torch.max(Y_pred_coarse_grain, 1)[1]

        test_fine_ground_truth += Y_fine_grain.tolist()
        test_coarse_ground_truth += Y_coarse_grain.tolist()

        test_fine_prediction += predicted_fine.tolist()
        test_coarse_prediction += predicted_coarse.tolist()

        name_list += names

Testing vit_b_16 on mps...


RuntimeError: MPS backend out of memory (MPS allocated: 3.20 GB, other allocations: 5.24 GB, max allowed: 9.07 GB). Tried to allocate 1.15 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
np.save()